### Init

Importe le module `cli` qui contient toute les commandes de l'API et configure l'API pour qu'elle se connecte au serveur.

In [1]:
import liv4dcli as cli
from pandas import DataFrame
from commands.utils import paste_int_list
cli.config.url = "http://liv4dfs.gi.polymtl.ca"

Créé des alias vers les ensembles de biomarqueusr et vers les ID d'utilisateur des cliniciens.

In [2]:
bright = "Exudates,Cotton Wool Spots,Drusen,Uncertain - Bright"
red = "Microaneurysms,Hemorrhages,Sub-retinal hemorrhage,Pre-retinal hemorrhage,Neovascularization,Uncertain - Red"
disk = "Disk,Cup,Macula"
vessel = "Vessels - Uncertain"
lesions = "Exudates,Cotton Wool Spots,Drusen,Uncertain - Bright,Microaneurysms,Hemorrhages,Sub-retinal hemorrhage,Pre-retinal hemorrhage,Neovascularization,Uncertain - Red"

boucher = "84be2a52-5491-4d67-ad75-1704964eeb51"
brent = "4207335d-4aea-4975-8ab0-3d46f8259e6f"
desroches = "d8fae448-3e1f-41cc-93cd-bef2df151d47"
duval = "02758e79-f1e1-4989-8e57-5658672a2b12"
hammamji = "7b1dc155-f885-4ad8-9582-b5db27196c66"
kalevar = "ed2f771f-fd43-4f0f-98f3-c42edfb5f3b4"
qian = "9dbc7fbd-66a9-4990-b53f-291f806f6672"
wong = "de7b71eb-87b0-4dce-84fe-373e86dc7eb8"


clinicians = [boucher, brent, desroches, duval, hammamji, kalevar, qian, wong]

clement = "e3252de6-e128-4553-91b7-ed392529cc7b"
gabriel = "b45c99c9-b10e-4cd9-96d7-4060a889184b"
philippe = "9b869c98-d334-426a-a824-7ca80c50e344"
demo = "a8d42b15-4d1e-4a95-a95e-852b9712fc71"

**Taches**
 - utilisateur
 - image
 - Complétée
 - Active (Visible)
 - Type de taches (toujours 1)
 
**Révisions (données annotations)**
 - Image
 - Utilisateur (une seule révision par utilisateur et par image)
 - Commentaire
     - Liste biomarqueurs modifiables
     - Diagnostique ou annotation de biomarqueurs (exclusif)
     - Timer
     - Commentaire affiché dans le champ commentaire
 - SVG (biomarqueurs en format xml puis png)
     
 

### Utilisateurs

Lister les utilisateurs:

In [ ]:
users = cli.user.list_user()
DataFrame.from_dict(users)

Ajouter un clinicien.

In [ ]:
#cli.user.create(name="name",email="email",password="password", role='clinician')

Modifier un mot de passe.

In [ ]:
#cli.user.update(gabriel, password='1234admin')

### Images

Liste des images par index: https://docs.google.com/spreadsheets/d/1AvLe63q63mK4RWOqEvJaUWy0_JMGgu07koFHr9chbQU/edit?usp=sharing.

Coller une liste d'index depuis le presse papier dans python:

In [ ]:
imgs = cli.utils.paste_int_list()
print(len(imgs),'|',imgs)

### Taches

Lister les taches d'un utilisateur:

In [ ]:
tasks = cli.task.list_task(boucher)
DataFrame.from_dict(tasks)

Ajoute une tache à un utilisateur:

In [ ]:
# image_id: L'identifiant de l'image ou une liste d'identifiant.
# user_id: L'identifiant de l'utilisateur
# limit_biomarkers: liste (ou string séparé par des virgules) des biomarqueurs modifiables
# active: Affiche la tache dans la liste des taches de l'utilisateur
# completed: Marque la tache comme complétée dans la liste des taches de l'utilisateur
# diagnostic: Remplace l'outils d'annotation de biomarqueurs par l'annotation du diagnostic (DR, MA)
# comment: Précise un commentaire 
# merge: Si une tache existe, elle est merge avec la nouvelle.

cli.task.create(10, demo)
cli.task.create([11,12], demo, limit_biomarkers=red)
cli.task.create(13, demo, diagnostic=True, comment="Faire le diagnostic")

Supprimer une tache d'un utilisateur:

Supprimer toutes les taches d'un utilisateur:

In [ ]:
cli.task.delete_all_from_user(demo)

### Revision

Lire une revision

In [ ]:
rev = cli.revision.get_revision(demo, 2)
print('ID', rev['id'])
print('Commentaire', rev['diagnostic'])
print('SVG', rev['svg'])

In [ ]:
rev = cli.revision.get_revision(demo, 11, svg=False)
print('Commentaire:', rev['diagnostic'])
rev = cli.revision.get_revision(demo, 13, svg=False)
print('Commentaire:', rev['diagnostic'])

Lire un biomarqueur

In [ ]:
#b = cli.revision.get_biomarkers(demo, 2)[vessel]
b = cli.revision.get_biomarker(user_id=demo, image_id=2, biomarker='Hemorrhages')

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(b,cmap='gray')
plt.axis('off')

Modifier une revision:

In [ ]:
#cli.revision.update(svg, demo, 2, "diagnostic")
cli.revision.update_diagnostic("diagnostic", demo, 2)


Modifier un biomarqueur:

In [ ]:
# img = cv2.imread(...)
png = cli.utils.encode_svg(img)
cli.revision.update_biomarker(demo, 2, vessel, png)

Supprimer tout les biomarqueurs

In [ ]:
#cli.revision.clear_revision(demo,2)

Transférer des biomarqueurs d'un clinicien à l'autre

In [ ]:
cli.revision.transfer_biomarker(2, demo, prod, biomarker=vessel) # Uniquement les vaisseaux
cli.revision.transfer_biomarker([10,11], demo, prod)  # Tous les biomarqueurs pour les images 0 et 11

# Révision par utilisateur

In [5]:
def revision_by_user(user):
    tasks = cli.task.list_task(user)
    return [cli.revision.get_revision(t['user']['id'], t['image']['id'], svg=False) for t in tasks]

def extract_time(revision):
    diagnostic_text = revision['diagnostic']
    return cli.utils.dict_info_from_diagnostic(diagnostic_text)['time']

In [4]:
rev_boucher = revision_by_user(boucher)
rev_duval = revision_by_user(duval)

In [14]:
times_boucher = np.array([extract_time(r) for r in rev_boucher])
avg_boucher = times_boucher.mean()
std_boucher = times_boucher.std()
print('Timer Bboucher:', times_boucher)
print('Avg: %i:%i +/- %i:%i' %
     (avg_boucher//60, avg_boucher%60, std_boucher//60, std_boucher%60))


Timer boucher [3415  710 1410  967  564 2433  329  618  598 1233 1099  717  659 1069
  396  488  379  307  200  379  228  259  217  397  717  721  363  799
  640  762  367 2166 1985 1142  268  692  630  335  636 1591  192  380
  396  704  227  750  717 1007 1264  365  521]
Avg: 12:52 +/- 10:18


In [16]:
times_duval = np.array([extract_time(r) for r in rev_duval])
avg_duval = times_duval.mean()
std_duval = times_duval.std()
print('Timer Duval:', times_boucher)
print('Avg: %i:%i +/- %i:%i' %
     (avg_duval//60, avg_duval%60, std_duval//60, std_duval%60))


Timer Duval: [3415  710 1410  967  564 2433  329  618  598 1233 1099  717  659 1069
  396  488  379  307  200  379  228  259  217  397  717  721  363  799
  640  762  367 2166 1985 1142  268  692  630  335  636 1591  192  380
  396  704  227  750  717 1007 1264  365  521]
Avg: 5:24 +/- 3:38
